Our first approach is to generate, by hand, a simple collections of sentences organised by intents and associated answers.
We will then process that data to make it suitable for NLP applications, encode it using "bag of word" and train a neural network to predict user intent from an utterance.

Then, we wil used pre-trained word embeddings

Then, we will generate training data using OpenAI GPT-3 API and train it, using both bag of words and word embeddings
We can also try TF-IDF to compare it with the NN.

# Import / Generate data

In [21]:
import json
import pandas as pd
from sklearn import tree, svm, naive_bayes
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
from time import time
from collections import defaultdict
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("en_core_web_lg")

In [ ]:
data_file = open('intents.json').read()
intents = json.loads(data_file)


data = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        data.append([pattern, intent['tag']])

df = pd.DataFrame(data, columns=['text','intent'])

# Data pre-processing

In [ ]:
# fine-tune preprocessing for spaCy word embeddings using this method : https://www.kaggle.com/code/christofhenkel/how-to-preprocessing-when-using-embeddings

In [ ]:
# Helper functions

def lemmatize_text(text, preprocessed=True):
    return process_text(text, "lemmatize", preprocessed)

def tokenize_text(text, preprocessed=True):
    return process_text(text, "tokenize", preprocessed)

def process_text(text, mode: str, preprocessed=True):
    if not preprocessed:
        text = nlp(text)
    if mode == "tokenize":
        processed_text = [token for token in text] # token and embed must have the same processing + SpaCy provides embeddings for punctuation
    elif mode == "embed":
        processed_text = [token.vector for token in text] # token and embed must have the same processing
    elif mode == "lemmatize":
        processed_text = [token.lemma_ for token in text
                               if not token.is_punct and not token.is_space and not token.like_url and not token.like_email]
    else:
        raise ValueError("Mode not supported")
    return processed_text

def save_preprocessed(raw_text, save_path):
    doc_bin = DocBin(attrs=["LEMMA", "ENT_IOB", "ENT_TYPE"], store_user_data=True)
    for doc in nlp.pipe(raw_text):
        doc_bin.add(doc)
    # save DocBin to a file on disc
    doc_bin.to_disk(save_path)

In [ ]:
file_name_spacy = 'preprocessed_documents.spacy'
#save_preprocessed(raw_text=df["text"], save_path="preprocessed_documents.spacy")

# Load DocBin at later time or on different system from disc or bytes object
doc_bin = DocBin().from_disk(file_name_spacy)
df["doc"] = list(doc_bin.get_docs(nlp.vocab))

# Data preparation

In [ ]:
X_train = df["doc"]
y_train = df["intent"]

vectorizer = TfidfVectorizer(ngram_range=(1, 1), lowercase=False, tokenizer=tokenize_text, max_features=3000)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_train_embedded = df["doc"].apply(process_text, args=("embed", True,))
X_train_embedded_avg = X_train_embedded.apply(sum).apply(pd.Series)

# The following code block construct a sentence representation as a sum of all embeddings of the words in it, weighted by their tfidf score

result = []
for (idxRow, sentence) in X_train.items():
    final = 0
    for idxWord, word in enumerate(sentence):
        tfidf_idx = vectorizer.vocabulary_[word]
        final += (X_train_tfidf.toarray())[idxRow][tfidf_idx] * X_train_embedded[idxRow][idxWord]
    result.append(final)

X_train_embedded_avg_tfidf = pd.Series(result).apply(pd.Series)

# Models preparation

In [63]:
vect = TfidfVectorizer(lowercase=False, tokenizer=lemmatize_text, max_features=3000)

# classifiers to use

# TODO : implement more classifiers

gs_dict_tfidf = defaultdict(dict)

dectree = tree.DecisionTreeClassifier()
svm_clf = svm.SVC()
multi_nb = naive_bayes.MultinomialNB() # Not suitable for negative values (thus not suitable for word embeddings)

gs_dict_tfidf['dectree']['pipeline'] = Pipeline([
    ('vect', vect),
    ('dectree', dectree)])
gs_dict_tfidf['svm_clf']['pipeline'] = Pipeline([
    ('vect', vect),
    ('svm_clf', svm_clf)])
gs_dict_tfidf['multi_nb']['pipeline'] = Pipeline([
    ('vect', vect),
    ('multi_nb', multi_nb)])

gs_dict_tfidf['dectree']['params'] = {
    "dectree__max_depth": [4, 40],
    "vect__ngram_range": ((1, 1), (1, 2), (1,3), (1,4)),
    "vect__use_idf": (True, False),
    "vect__binary": (True, False),
}
gs_dict_tfidf['svm_clf']['params'] = {
    "svm_clf__kernel": ["linear", "rbf"],
    "vect__ngram_range": ((1, 1), (1, 2), (1,3), (1,4)),
    "vect__use_idf": (True, False),
    "vect__binary": (True, False),
}
gs_dict_tfidf['multi_nb']['params'] = {
    "multi_nb__alpha": [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000],
    "vect__ngram_range": ((1, 1), (1, 2), (1,3), (1,4)),
    "vect__use_idf": (True, False),
    "vect__binary": (True, False),
}

^Classic below : WORD EMBEDDINGS

In [64]:
gs_dict_embeddings = defaultdict(dict)
# classifiers to use
dectree = tree.DecisionTreeClassifier()
svm_clf = svm.SVC()

gs_dict_embeddings['dectree']['pipeline'] = Pipeline([
    ('dectree', dectree)])
gs_dict_embeddings['svm_clf']['pipeline'] = Pipeline([
    ('svm_clf', svm_clf)])

gs_dict_embeddings['dectree']['params'] = {
    "dectree__max_depth": [4, 10],
}
gs_dict_embeddings['svm_clf']['params'] = {
    "svm_clf__kernel": ["linear", "rbf"],
}

# Model Selection

In [59]:
# Helper functions

def perform_grid_search(X_train, y_train, pipeline, parameters, scoring):
    gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1, verbose=1, cv=3, scoring=scoring) # Issue when n_jobs = -1 OR > 1
    # I believe this may be because we use a custom tokenizer in TfidfVectorizer(), can't find how to solve it
    print("\n------------------------------------------------------------------------\n")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()

    gs_clf.fit(X_train, y_train)

    print("\nDone in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % gs_clf.best_score_)
    print("Best parameters set:")
    best_parameters = gs_clf.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print(f"\t'{param_name}': '{best_parameters[param_name]}'")
    return gs_clf

def best_estimator_per_clf(X_train, y_train, gs_dict: defaultdict, scoring):
    for clf in dict(gs_dict):
        gs_dict[clf]['gs'] = perform_grid_search(
            X_train,
            y_train,
            gs_dict[clf]['pipeline'],
            gs_dict[clf]['params'],
            scoring
        )

Traditional classifiers cannot accept as input a sequence of word embeddings, so we try 2 strategies :
- Averaging all embeddings in a sentence
- Averaging using TF-IDF score as weight

In [68]:
best_estimator_per_clf(X_train_embedded_avg_tfidf, y_train, gs_dict_embeddings, scoring="accuracy")

# TODO : implement multiple scoring : https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py

# TODO : extract metrics https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py


------------------------------------------------------------------------

pipeline: ['dectree']
parameters:
{'dectree__max_depth': [4, 10]}
Fitting 3 folds for each of 2 candidates, totalling 6 fits

Done in 0.066s

Best score: 0.421
Best parameters set:
	'dectree__max_depth': '10'

------------------------------------------------------------------------

pipeline: ['svm_clf']
parameters:
{'svm_clf__kernel': ['linear', 'rbf']}
Fitting 3 folds for each of 2 candidates, totalling 6 fits

Done in 0.042s

Best score: 0.739
Best parameters set:
	'svm_clf__kernel': 'linear'


In [74]:
test = sum(process_text(nlp("I want to print 76 page of a document"), mode="embed"))
test

array([ 4.61637974e-01,  1.13560700e+00, -1.02110004e+00,  8.33649635e-02,
        5.00504971e-01,  2.61418998e-01, -3.87266040e-01, -1.54758000e+00,
        5.03413826e-02,  1.57266893e+01, -2.69594002e+00,  9.97252107e-01,
        1.69486105e+00, -4.74574983e-01, -1.32837963e+00, -4.22390997e-01,
       -8.19185078e-01,  1.50653019e+01, -1.59457624e+00, -1.56144702e+00,
        6.65617049e-01, -8.29769969e-02, -2.71458983e-01,  2.03112975e-01,
        6.40709996e-01,  1.32894230e+00, -1.56676382e-01, -2.17072773e+00,
       -3.94010007e-01,  4.64303017e-01,  5.08952022e-01,  1.70764005e+00,
        8.82860124e-02, -5.47645867e-01, -8.17725062e-01, -2.17576098e+00,
       -1.03687799e+00,  3.65506977e-01,  2.05558538e-03, -7.48545110e-01,
        1.99170977e-01,  1.29109299e+00,  1.56570300e-02, -2.27649307e+00,
       -9.02953029e-01,  2.60586470e-01, -3.59949052e-01,  1.38292694e+00,
        8.08113277e-01,  8.23220015e-02, -2.31747699e+00,  1.25209391e-02,
        9.51222003e-01, -

# Model evaluation

skew text classification if name entities are found (either by multiple channels NN or by adding a feature to the data passed)

In [76]:
# LSTM : https://www.tensorflow.org/text/tutorials/text_classification_rnn